In [1]:
# 定义一个模板字符串，这个模板将用于生成提问
template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""

# Question: is template used below?

In [2]:
# 定义一个Pydantic数据格式，这个格式描述了一个"行动"类及其属性
from pydantic import BaseModel, Field
class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")

In [3]:
# 使用Pydantic格式Action来初始化一个输出解析器
from langchain.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=Action)

下面是对 `PromptTemplate` 的 `input_variables` 和 `partial_variables` 参数的简单解释：

1. **input_variables**:
   - 这是一个包含变量名的列表，用于指定在提示模板中需要动态替换的部分。
   - 模板中的这些变量将由用户在运行时提供的值替换。
   - 例如，如果模板是 "Hello, {name}!"，那么 `input_variables` 就应该包含 "name"，以便你在使用模板时可以提供一个具体的名字。
2. **partial_variables**:
   - 这是一个包含部分变量名和其对应值的字典，用于提前替换模板中的某些部分。
   - 这些变量的值是在模板创建时就固定的，不需要在运行时再提供。
   - 例如，如果模板是 "Hello, {name}! Today is {day}."，并且你已经知道今天是周一，你可以将 `day` 设置为 "Monday" 作为 `partial_variables` 的一部分。这样，在使用模板时只需提供 `name` 的值。

***举个例子：***

假设我们有以下模板：

```python
template = "Hello, {name}! Today is {day}."
```
- input_variables: ["name"]
  - 这意味着在使用模板时，我们需要提供 name 的值。
- partial_variables: {"day": "Monday"}
  - 这意味着 day 的值已经固定为 "Monday"，不需要在运行时提供。

使用模板时：
```python
prompt = PromptTemplate(
    template="Hello, {name}! Today is {day}.",
    input_variables=["name"],
    partial_variables={"day": "Monday"}
)

formatted_prompt = prompt.format_prompt(name="Alice")
```

结果：
formatted_prompt 的值将是 "Hello, Alice! Today is Monday."

这样，input_variables 和 partial_variables 就帮助我们动态和静态地替换模板中的变量。


In [8]:
# 定义一个提示模板，它将用于向模型提问
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_value = prompt.format_prompt(query="What are the colors of Orchid?")
prompt_value

StringPromptValue(text='Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"action": {"description": "action to take", "title": "Action", "type": "string"}, "action_input": {"description": "input to the action", "title": "Action Input", "type": "string"}}, "required": ["action", "action_input"]}\n```\nWhat are the colors of Orchid?\n')

In [5]:
# 定义一个错误格式的字符串
bad_response = '{"action": "search"}'
parser.parse(bad_response) # 如果直接解析，它会引发一个错误

OutputParserException: Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [6]:
# 尝试用OutputFixingParser来解决这个问题
import os
from langchain.output_parsers import OutputFixingParser
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("BASE_URL"),
    openai_api_version=os.getenv("API_VERSION"),
    deployment_name=os.getenv("DEPLOYMENT_NAME"),
    openai_api_key=os.getenv("API_KEY"),
)

fix_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
parse_result = fix_parser.parse(bad_response)
print('OutputFixingParser的parse结果:', parse_result)

OutputFixingParser的parse结果: action='search' action_input='example input'


- 解决的问题有：
  - 不完整的数据：原始的 bad_response 只提供了 action 字段而没有 action_input 字段。OutputFixingParser 已经填补了这个缺失，为 action_input 字段提供了值 'query'。
- 没解决的问题有：
  - 具体性：尽管 OutputFixingParser 为 action_input 字段提供了默认值 'query'，但这并不具有描述性。真正的查询是 “Orchid（兰花）的颜色是什么？”。所以，这个修复只是提供了一个通用的值，并没有真正地回答用户的问题。
  - 可能的误导：'query' 可能被误解为一个指示，要求进一步查询某些内容，而不是作为实际的查询输入。

In [7]:
# 初始化RetryWithErrorOutputParser，它会尝试再次提问来得到一个正确的输出
from langchain.output_parsers import RetryWithErrorOutputParser

llm2 = AzureChatOpenAI(
    azure_endpoint=os.getenv("BASE_URL"),
    openai_api_version=os.getenv("API_VERSION"),
    deployment_name=os.getenv("DEPLOYMENT_NAME"),
    openai_api_key=os.getenv("API_KEY"),
    temperature=0
)

retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=llm2
)
parse_result = retry_parser.parse_with_prompt(bad_response, prompt_value)
print('RetryWithErrorOutputParser的parse结果:',parse_result)

RetryWithErrorOutputParser的parse结果: action='search' action_input='colors of Orchid'
